# Results 
- Plot training/test telemetry values, predictions, smoothed errors, and predicted and actual anomalies
- A specified results file from the `results` dir is used to highlight anomalous regions and read in data from the `data` dir

In [ ]:
import sys
import numpy as np
import os
sys.path.insert(0, '..')
from telemanom.plotting import Plotter
import pandas as pd
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode
import cufflinks as cf
import glob
cf.go_offline()
init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Select set of results to visualize

In [ ]:
# Default to most recent
results_fn = glob.glob('../results/*.csv')[-1]
print('Using most recent run: {}'.format(results_fn.split("/")[-1]))

run_id = results_fn.split("/")[-1][:-4]

## Parameters
- See `config.yaml` for parameter explanations

In [ ]:
with open('../data/{}/params.log'.format(run_id), 'r') as f:
    for i,row in enumerate(f.readlines()):
        if len(row) < 2:
            break
        if i > 0:
            print(' '.join(row.split(' ')[3:]).replace('\n',''))

## Summary of results

In [ ]:
plotter = Plotter(run_id)
plotter.all_result_summary()

## Interactive inline Plotly charts for viewing `y_test`, `y_hat`, and `smoothed errors (e_s)`
- **Blue** highlighted regions indicate anomalous sequences detected by the system
- If available, **Red** highlighted regions indicate true anomalous regions
- Can also optionally plot training data by setting `plot_train=True`

In [ ]:
# Get channels from the project configuration
sys.path.insert(0, os.path.abspath('../../helpers'))
from helpers import (
    get_projects_and_run_settings,
    load_config
)

prj_config = load_config('../../helpers/prj_config.yaml')
projects, run_settings = get_projects_and_run_settings(prj_config)

example_project = None
for project in projects:
    if run_settings['project'] in project:
        example_project = project[run_settings['project']]

if example_project is None:
    print(f"[anomaly_aie_lstm] Project name {run_settings['project']}' not found in prj_config.yaml.")
else:
    for model_par in example_project['graphs']:
        for i in range(0, len(model_par['models'])):
            plotter.plot_channel(model_par['models'][i], plot_errors=True, plot_train=False)